In [1]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option("display.max_columns", None)

data_modeling.h5 is the dataset merged and cleaned in the previous notebook.

In [2]:
store = pd.HDFStore('df_all_frac.h5')
data = store['df_all_frac']

In [3]:
data

,unique_id,loan_status,report_date,mob,bom,ppmt,ipmt,co_amt,prepay_amt,eom,prepay,default,loan_amount,monthly_payment,grade,income,inq_last_6m,num_open_accts,revol_bal,num_tot_accts,tot_credit_bal,accts_opn_last_6m,inq_last_12m,accts_opn_last_24m,employment_length,home_ownership,int_rate,earliest_cr_line,fico,util_rate,loan_over_income,term,dti,platform_id,Orig_year,Orig_month,post_dti
0,A-472465-238102,late_31_120_days,OCT2018,9,17859.707244,0.000000,0.000000,0.0,0.0,17859.707244,0,0,19200.0,457.28,C,60000.0000,0.745387,17.000000,5057.000000,22.000000,17039.000000,1.000000,1.000000,8.000000,1,RENT,15.05,197.000000,707.0,14.200000,0.320000,60,0.172900,A,2018,1,0.264354
1,A-242571-52012,current,OCT2018,10,7394.570057,244.210483,64.209517,0.0,0.0,7150.359574,0,0,9500.0,308.42,B,56000.0000,1.000000,8.000000,427.000000,23.000000,28382.000000,2.000000,1.000000,3.000000,3,RENT,10.42,212.000000,707.0,21.400000,0.170000,36,0.225400,A,2017,12,0.291489
2,A-412526-60744,current,APR2018,1,15000.000000,373.751667,131.508333,0.0,0.0,14626.248333,0,0,15000.0,465.50,A,140000.0000,0.745387,9.000000,28695.000000,18.000000,55504.000000,29.927927,1.000000,4.593272,8,MORTGAGE/OWN,7.34,210.000000,722.0,82.500000,0.110000,36,0.134800,A,2018,2,0.174700
3,A-55799-190157,current,SEP2018,9,17657.245603,208.676735,315.623265,0.0,0.0,17448.568868,0,0,19200.0,524.30,D,82000.0000,0.745387,19.000000,15882.000000,29.000000,39287.000000,1.000000,1.000000,6.000000,6,RENT,21.45,110.000000,672.0,41.900000,0.230000,60,0.206200,A,2018,1,0.282926
4,A-604196-70503,current,DEC2019,20,5109.849264,280.282058,44.327942,0.0,0.0,4829.567207,0,0,10000.0,324.61,B,65000.0000,1.000000,8.000000,1356.000000,20.000000,4143.000000,1.000000,2.000000,2.000000,6,RENT,10.41,213.000000,682.0,7.900000,0.150000,36,0.106400,A,2018,4,0.166327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10381408,C-582921-26509,current,201905,23,17827.650000,376.150000,208.800000,0.0,0.0,17451.500000,0,0,25000.0,584.95,C,60000.0000,0.745387,15.000000,7919.000000,28.000000,255.000000,29.927927,2.363403,1.000000,5,RENT,14.25,2.000000,670.0,45.000000,0.416667,60,0.233012,C,2017,6,0.350000
10381409,C-687454-163583,current,201903,9,2000.000000,86.120000,8.730000,0.0,0.0,1913.880000,0,0,3150.0,94.85,AA,156000.0000,0.745387,1.000000,3353.000000,3.000000,194476.000000,29.927927,2.363403,2.000000,3,MORTGAGE/OWN,5.31,182.000000,830.0,5.000000,0.020192,36,0.002704,C,2018,6,0.010000
10381410,C-972909-1641,current,201909,44,11539.690000,589.490000,110.260000,0.0,0.0,10950.200000,0,0,32000.0,699.75,B,69999.9996,0.745387,11.074699,14571.576638,21.478623,133845.268705,29.927927,2.363403,4.593272,3,NoData,11.25,179.026931,750.0,46.936703,0.457143,60,0.110045,C,2016,1,0.230000
10381411,C-256514-14846,current,201703,3,5692.150000,157.460000,23.230000,0.0,0.0,5534.690000,0,0,6000.0,180.69,AA,39999.9996,0.745387,11.074699,14571.576638,21.478623,133845.268705,29.927927,2.363403,4.593272,10,NoData,5.32,179.026931,790.0,46.936703,0.150000,36,0.065794,C,2016,12,0.120000


Remove report_date

In [4]:
data.drop('report_date', inplace=True, axis=1)

Perform Treatment to the dti


In [5]:
X = data.drop(['default', 'prepay','unique_id', 'loan_status','bom','eom','ppmt','ipmt','prepay_amt','co_amt'], axis=1)
X_mob = X['mob']
X_year = X['Orig_year']
X_month = X['Orig_month']

X = X.drop('mob', axis=1)
X = X.drop('Orig_year', axis=1)
X = X.drop('Orig_month', axis=1)

y_default = data['default']
y_prepay = data['prepay']

In [6]:
encoder_1 = LabelEncoder()

# Fit the encoder on the data "platform_id"
encoder_1.fit(X["platform_id"])
X_platform_id_encoded = encoder_1.transform(X["platform_id"])
X["platform_id"] = X_platform_id_encoded

# Fit the encoder on the data "grade"
encoder_2 = LabelEncoder()
encoder_2.fit(X["grade"])
X_grade_encoded = encoder_2.transform(X["grade"])
X["grade"] = X_grade_encoded

In [7]:
# Fit the encoder on the data "home_ownership"
encoder_3 = LabelEncoder()
encoder_3.fit(X["home_ownership"])
X_home_ownership_encoded = encoder_3.transform(X["home_ownership"])
X["home_ownership"] = X_home_ownership_encoded

In [11]:
# Do feature scaling
sc = StandardScaler()
X_scaled = sc.fit_transform(X)

# Apply PCA
pca = PCA(n_components= 15)
X_pca = pca.fit_transform(X_scaled)

# Get the transformed dataset
X_pca = pd.DataFrame(X_pca)
print(X_pca.head())
print("\nSize: ")
print(X_pca.shape)

         0         1         2         3         4         5         6   \
0  0.093127  0.092207  0.494277 -0.502171  2.210740  0.645226  0.518809   
1 -1.148829 -1.523952  0.578549  0.395206  0.826511  0.892112  0.247240   
2  0.630276 -1.859233 -1.037141  1.116298 -0.983651  0.455403 -0.271425   
3  0.150906  1.540984 -0.652540  0.263882  2.258937  0.715997 -0.127785   
4 -1.312144 -1.485046  0.662994 -0.109456  0.347366  0.839628 -0.036324   

         7         8         9         10        11        12        13  \
0 -1.251009  0.102798 -0.155370 -1.872736  0.456408 -0.436088 -0.204955   
1 -0.727809  0.032526  0.343827 -0.395095  0.183804 -0.079973  0.727229   
2  0.416954 -0.052584  0.182261 -0.026139  0.102932 -0.579591  0.751029   
3  0.060508  0.023202 -0.311532 -1.664690  0.349264 -0.558741 -0.022529   
4 -0.188709 -0.011632  0.355584 -0.611471  0.298834 -0.938122  0.889392   

         14  
0  0.040391  
1 -0.231855  
2  0.066680  
3  0.720693  
4 -0.004444  

Size: 
(10381

In [12]:
store = pd.HDFStore('X_frac_pca.h5')
store['X_frac_pca'] = X_pca  # save it

store = pd.HDFStore('X_frac_mob.h5')
store['X_frac_mob'] = X_mob  # save it

store = pd.HDFStore('y_frac_default.h5')
store['y_frac_default'] = y_default  # save it

store = pd.HDFStore('y_frac_prepay.h5')
store['y_frac_prepay'] = y_prepay  # save it

store = pd.HDFStore('X_frac_year.h5')
store['X_frac_year'] = X_year  # save it

store = pd.HDFStore('X_frac_month.h5')
store['X_frac_month'] = X_month  # save it

In [13]:
store = pd.HDFStore('X_frac.h5')
store['X_frac'] = X  # save it